In [1]:
from collections import Counter
from cltk.stem.lemma import LemmaReplacer
from cltk.stem.latin.j_v import JVReplacer
import cltk.corpus.latin

jv_replacer = JVReplacer()

skip_chars = '0123456789`~!@#$%^&*()))_+-={}[]|\:;"\'<>?,./'

def clean_text(text):
    table = {ord(char): None for char in skip_chars}
    return text.translate(table)

nonword_name_abbreviations = 'Ap,A,D,F,C,Cn,H,L,Mai,Mam,M,Min,N,Oct,P,Q,Seq,Ser,Sp,St,T,Ti,V,Vol,Vop'.lower().split(',')

def clean_entry(entry):
    if (entry in nonword_name_abbreviations):
        return entry.capitalize()+'.'
    return ''.join(filter(lambda x: x.isalpha(), entry))

def get_corpus_files(author):
    fileids = filter(lambda s: s.startswith(author), cltk.corpus.latin.latinlibrary.fileids())
    files = [cltk.corpus.latin.latinlibrary.abspath(fileid) for fileid in fileids]
    return files

def merge_file_contents(paths):
    text = ''
    for path in paths:
        f = open(path)
        text += f.read()
        #print('loaded', path, 'buffer length', len(text))
        f.close()
    return text


In [2]:
# We need to import a data model to train the lemmatizer.

import os
from cltk.utils.file_operations import open_pickle

# Set up training sentences

rel_path = os.path.join('~/cltk_data/latin/model/latin_models_cltk/lemmata/backoff')
path = os.path.expanduser(rel_path)

# Check for presence of latin_pos_lemmatized_sents
file = 'latin_pos_lemmatized_sents.pickle'      

latin_pos_lemmatized_sents_path = os.path.join(path, file)
if os.path.isfile(latin_pos_lemmatized_sents_path):
    latin_pos_lemmatized_sents = open_pickle(latin_pos_lemmatized_sents_path)
else:
    latin_pos_lemmatized_sents = []
    print('The file %s is not available in cltk_data' % file)  
    
# Set up CLTK Latin backoff lemmatizer
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
lemmatizer = BackoffLatinLemmatizer(latin_pos_lemmatized_sents)

In [3]:
from cltk.tokenize.word import WordTokenizer

authors = (
    'caesar',
    'livy',
    'ovid',
    'bacon',
    'jerome',
    'newton'
)
word_tokenizer = WordTokenizer('latin')

data = {}
for author in authors:
    freq_table = []

    files = get_corpus_files(author)
    text = merge_file_contents(files)
    text = clean_text(text)

#     tokens = word_tokenizer.tokenize(jv_replacer.replace(text).lower())
    tokens = word_tokenizer.tokenize(text.lower())
    lemmataPairs = lemmatizer.lemmatize(tokens)
    lemmata = [pair[1] for pair in lemmataPairs]
    N = len(lemmata)
    longest_token = max(lemmata, key=len)
    freq_count = Counter(lemmata)
    data[author] = freq_count

In [6]:
def percent(n, decimal=2):
    return round(n * 100, decimal)

def normalize(freq_table):
    result = freq_table.copy()
    N = sum(freq_table.values())
    for lemma in result:
        result[lemma] /= N
    return result

def normalize_all(data):
    result = data.copy()
    for author in result:
        result[author] = normalize(result[author])
    return result

def merge_all(data):
    result = Counter()
    for author in data:
        result += data[author]
    return result

def word_use_detail(word, author):
    author_data = data[author]
    N = sum(author_data.values())
    return author[0:3] + ':' + str(percent(author_data[word] / N, 2))

PERIOD = 1
LAST_WORD = 50000
SHOW_DETAILS = False

def print_counter_data(counter):
    f = open('+'.join(authors)+'-'+str(LAST_WORD)+'.txt', 'w')
    coverage = 0
    N = sum(counter.values())
    index = 1
    for word, freq in counter.most_common(LAST_WORD):
        coverage += freq / N
        if index == 1 or index % PERIOD == 0:

            details = ' '.join([word_use_detail(word, author) for author in authors])
            segs = (
                index,
                clean_entry(word),
                percent(freq, 4),
                percent(coverage, 1),
                details if SHOW_DETAILS else '',
            )
            line = ' '.join((
                '%5s' % segs[0],
                '%-15s' % segs[1],
                '%7s' % segs[2],
                '(%5s)' % segs[3],
                segs[4],
            ))
            print(line)
            f.write('\t'.join([str(seg) for seg in segs])+'\n')
        index += 1
    f.close()
normalized_combined_freq_table = normalize(merge_all(data))
print_counter_data(normalized_combined_freq_table)

    1 sum              3.1819 (  3.2) 
    2 et               2.6938 (  5.9) 
    3 in               2.1867 (  8.1) 
    4 qui              2.1721 ( 10.2) 
    5 que              2.0896 ( 12.3) 
    6 is               1.3741 ( 13.7) 
    7 ad               1.1131 ( 14.8) 
    8 cum              1.0468 ( 15.9) 
    9 ab               0.8812 ( 16.7) 
   10 ut               0.8642 ( 17.6) 
   11 non              0.7725 ( 18.4) 
   12 hic               0.763 ( 19.1) 
   13 sui              0.6811 ( 19.8) 
   14 suus             0.5871 ( 20.4) 
   15 neque            0.5857 ( 21.0) 
   16 atque             0.557 ( 21.5) 
   17 omnis            0.5504 ( 22.1) 
   18 ex               0.5127 ( 22.6) 
   19 res               0.504 ( 23.1) 
   20 quam             0.4834 ( 23.6) 
   21 ille              0.466 ( 24.1) 
   22 facio            0.4335 ( 24.5) 
   23 ipse             0.4323 ( 24.9) 
   24 possum           0.4058 ( 25.3) 
   25 si               0.4028 ( 25.7) 
   26 magnus           0.

  588 peruenio         0.0247 ( 65.8) 
  589 sex              0.0246 ( 65.8) 
  590 aeger            0.0246 ( 65.8) 
  591 qual             0.0246 ( 65.9) 
  592 contendo         0.0245 ( 65.9) 
  593 existimo         0.0245 ( 65.9) 
  594 repeto           0.0245 ( 65.9) 
  595 factum           0.0244 ( 66.0) 
  596 occurro          0.0244 ( 66.0) 
  597 amplus           0.0244 ( 66.0) 
  598 supero           0.0244 ( 66.0) 
  599 ostendo          0.0242 ( 66.1) 
  600 forum            0.0242 ( 66.1) 
  601 oculus           0.0241 ( 66.1) 
  602 appareo          0.0241 ( 66.1) 
  603 numen            0.0241 ( 66.2) 
  604 impedio           0.024 ( 66.2) 
  605 motus             0.024 ( 66.2) 
  606 doceo            0.0239 ( 66.2) 
  607 manlius          0.0239 ( 66.3) 
  608 memoria          0.0238 ( 66.3) 
  609 vultus           0.0238 ( 66.3) 
  610 postulo          0.0237 ( 66.3) 
  611 tego             0.0237 ( 66.4) 
  612 remitto          0.0236 ( 66.4) 
  613 juvenis          0.

 1607 daris            0.0084 ( 80.2) 
 1608 luctus           0.0084 ( 80.2) 
 1609 fortasse         0.0084 ( 80.2) 
 1610 crassus          0.0084 ( 80.2) 
 1611 vulgus           0.0084 ( 80.3) 
 1612 depopulor        0.0084 ( 80.3) 
 1613 fundo            0.0084 ( 80.3) 
 1614 puppis           0.0084 ( 80.3) 
 1615 insum            0.0084 ( 80.3) 
 1616 quinctio         0.0084 ( 80.3) 
 1617 venus            0.0084 ( 80.3) 
 1618 penna            0.0084 ( 80.3) 
 1619 moror            0.0083 ( 80.3) 
 1620 vasto            0.0083 ( 80.3) 
 1621 laetitia         0.0083 ( 80.3) 
 1622 totiens          0.0083 ( 80.3) 
 1623 raptim           0.0083 ( 80.4) 
 1624 actio            0.0083 ( 80.4) 
 1625 juro             0.0083 ( 80.4) 
 1626 seruius          0.0083 ( 80.4) 
 1627 interior         0.0083 ( 80.4) 
 1628 dives            0.0083 ( 80.4) 
 1629 funus            0.0083 ( 80.4) 
 1630 seruo            0.0083 ( 80.4) 
 1631 tormentum        0.0083 ( 80.4) 
 1632 intermitto       0.

 2607 eximo            0.0044 ( 86.3) 
 2608 quatio           0.0044 ( 86.3) 
 2609 stella           0.0044 ( 86.3) 
 2610 observatio       0.0044 ( 86.3) 
 2611 substantia       0.0044 ( 86.3) 
 2612 notitia          0.0043 ( 86.3) 
 2613 artus            0.0043 ( 86.3) 
 2614 superaris        0.0043 ( 86.3) 
 2615 certo            0.0043 ( 86.3) 
 2616 mediocris        0.0043 ( 86.3) 
 2617 reliquiae        0.0043 ( 86.3) 
 2618 comminus         0.0043 ( 86.3) 
 2619 velis            0.0043 ( 86.3) 
 2620 cliens           0.0043 ( 86.3) 
 2621 dirigo           0.0043 ( 86.3) 
 2622 remus            0.0043 ( 86.3) 
 2623 exsto            0.0043 ( 86.3) 
 2624 theatrum         0.0043 ( 86.3) 
 2625 novitas          0.0043 ( 86.3) 
 2626 nitor            0.0043 ( 86.3) 
 2627 odor             0.0043 ( 86.3) 
 2628 que              0.0043 ( 86.3) 
 2629 lustro           0.0043 ( 86.3) 
 2630 octoginta        0.0043 ( 86.4) 
 2631 coitus           0.0043 ( 86.4) 
 2632 castus           0.

 3609 algidus          0.0026 ( 89.6) 
 3610 celtiberi        0.0026 ( 89.6) 
 3611 obstino          0.0026 ( 89.6) 
 3612 locri            0.0026 ( 89.6) 
 3613 epulae           0.0026 ( 89.6) 
 3614 exitium          0.0026 ( 89.6) 
 3615 cethegus         0.0026 ( 89.6) 
 3616 uelum            0.0026 ( 89.6) 
 3617 marmoreus        0.0026 ( 89.6) 
 3618 insero           0.0026 ( 89.6) 
 3619 invidiosus       0.0026 ( 89.6) 
 3620 curvus           0.0026 ( 89.6) 
 3621 vacca            0.0026 ( 89.6) 
 3622 madeo            0.0026 ( 89.6) 
 3623 notus            0.0026 ( 89.6) 
 3624 rebellis         0.0026 ( 89.6) 
 3625 henrico          0.0026 ( 89.6) 
 3626 instantiis       0.0026 ( 89.6) 
 3627 aegypti          0.0025 ( 89.6) 
 3628 specus           0.0025 ( 89.6) 
 3629 descio           0.0025 ( 89.6) 
 3630 quadriremis      0.0025 ( 89.6) 
 3631 dominatio        0.0025 ( 89.6) 
 3632 exsisto          0.0025 ( 89.6) 
 3633 illyricus        0.0025 ( 89.6) 
 3634 vigilo           0.

...

30115 lureconsultos    0.0001 ( 99.5) 
30116 decoraris        0.0001 ( 99.5) 
30117 subidos          0.0001 ( 99.5) 
30118 neapolitanumm    0.0001 ( 99.5) 
30119 subjugo          0.0001 ( 99.5) 
30120 stringendo       0.0001 ( 99.5) 
30121 angeovinaris     0.0001 ( 99.5) 
30122 arbitrui         0.0001 ( 99.5) 
30123 dispersit        0.0001 ( 99.5) 
30124 pisanorum        0.0001 ( 99.5) 
30125 sforza           0.0001 ( 99.5) 
30126 dirupit          0.0001 ( 99.5) 
30127 despicire        0.0001 ( 99.5) 
30128 destinebat       0.0001 ( 99.5) 
30129 orginal          0.0001 ( 99.5) 
30130 augustinum       0.0001 ( 99.5) 
30131 bardicum         0.0001 ( 99.5) 
30132 venetiaris       0.0001 ( 99.5) 
30133 sforzam          0.0001 ( 99.5) 
30134 medliolani       0.0001 ( 99.5) 
30135 feodum           0.0001 ( 99.5) 
30136 cecilia          0.0001 ( 99.5) 
30137 barkhamsted      0.0001 ( 99.5) 
30138 foderinghamiam   0.0001 ( 99.5) 
30139 praecelleret     0.0001 ( 99.5) 
30140 accumulasse      0.

31067 mathematices     0.0001 ( 99.6) 
31068 optica           0.0001 ( 99.6) 
31069 dotetur          0.0001 ( 99.6) 
31070 meritorius       0.0001 ( 99.6) 
31071 redditio         0.0001 ( 99.6) 
31072 aberatio         0.0001 ( 99.6) 
31073 erroneam         0.0001 ( 99.6) 
31074 imperviam        0.0001 ( 99.6) 
31075 perculserit      0.0001 ( 99.6) 
31076 inartificiosum   0.0001 ( 99.6) 
31077 facessentes      0.0001 ( 99.6) 
31078 pertentando      0.0001 ( 99.6) 
31079 aberraverint     0.0001 ( 99.6) 
31080 intricando       0.0001 ( 99.6) 
31081 detriminatis     0.0001 ( 99.6) 
31082 etvix            0.0001 ( 99.6) 
31083 gradaevitas      0.0001 ( 99.6) 
31084 juniori          0.0001 ( 99.6) 
31085 juvene           0.0001 ( 99.6) 
31086 increbuerunt     0.0001 ( 99.6) 
31087 patuerint        0.0001 ( 99.6) 
31088 illustrati       0.0001 ( 99.6) 
31089 authori          0.0001 ( 99.6) 
31090 ligaverint       0.0001 ( 99.6) 
31091 maleficiati      0.0001 ( 99.6) 
31092 subjecerit       0.

32242 espinetis        0.0001 ( 99.7) 
32243 inquisitus       0.0001 ( 99.7) 
32244 extasis          0.0001 ( 99.7) 
32245 incausabilia     0.0001 ( 99.7) 
32246 consistentibus   0.0001 ( 99.7) 
32247 circulatio       0.0001 ( 99.7) 
32248 pertransitio     0.0001 ( 99.7) 
32249 provehuntur      0.0001 ( 99.7) 
32250 adunaris         0.0001 ( 99.7) 
32251 contemperat      0.0001 ( 99.7) 
32252 commiscebat      0.0001 ( 99.7) 
32253 condensatis      0.0001 ( 99.7) 
32254 vitriolo         0.0001 ( 99.7) 
32255 jugum            0.0001 ( 99.7) 
32256 consequutio      0.0001 ( 99.7) 
32257 proprioris       0.0001 ( 99.7) 
32258 circulariter     0.0001 ( 99.7) 
32259 justior          0.0001 ( 99.7) 
32260 sclopetor        0.0001 ( 99.7) 
32261 perscrutantibus  0.0001 ( 99.7) 
32262 irrequiete       0.0001 ( 99.7) 
32263 pulsus           0.0001 ( 99.7) 
32264 decubitus        0.0001 ( 99.7) 
32265 imaginativum     0.0001 ( 99.7) 
32266 vellicentur      0.0001 ( 99.7) 
32267 provocentur      0.

33072 aspirent         0.0001 ( 99.8) 
33073 volitantium      0.0001 ( 99.8) 
33074 inscrutabilis    0.0001 ( 99.8) 
33075 praedominantis   0.0001 ( 99.8) 
33076 exploratu        0.0001 ( 99.8) 
33077 adiiciant        0.0001 ( 99.8) 
33078 mechanicam       0.0001 ( 99.8) 
33079 excellentiam     0.0001 ( 99.8) 
33080 pulsandae        0.0001 ( 99.8) 
33081 cytharae         0.0001 ( 99.8) 
33082 domitianus       0.0001 ( 99.8) 
33083 collimandis      0.0001 ( 99.8) 
33084 caracalla        0.0001 ( 99.8) 
33085 aurigatio        0.0001 ( 99.8) 
33086 incredible       0.0001 ( 99.8) 
33087 exhilararis      0.0001 ( 99.8) 
33088 consistendo      0.0001 ( 99.8) 
33089 subiugandis      0.0001 ( 99.8) 
33090 retrogradam      0.0001 ( 99.8) 
33091 melancholicus    0.0001 ( 99.8) 
33092 dioclesiano      0.0001 ( 99.8) 
33093 consuervit       0.0001 ( 99.8) 
33094 obicem           0.0001 ( 99.8) 
33095 apollonii        0.0001 ( 99.8) 
33096 distendebat      0.0001 ( 99.8) 
33097 relaxabat        0.

34088 secundariam      0.0001 ( 99.9) 
34089 iniiciant        0.0001 ( 99.9) 
34090 consulerint      0.0001 ( 99.9) 
34091 deliberato       0.0001 ( 99.9) 
34092 destituendi      0.0001 ( 99.9) 
34093 iniquiori        0.0001 ( 99.9) 
34094 demanda          0.0001 ( 99.9) 
34095 eiumodi          0.0001 ( 99.9) 
34096 qualemqual       0.0001 ( 99.9) 
34097 potuissit        0.0001 ( 99.9) 
34098 remuneretur      0.0001 ( 99.9) 
34099 oscitanter       0.0001 ( 99.9) 
34100 demandes         0.0001 ( 99.9) 
34101 iteratio         0.0001 ( 99.9) 
34102 aequipollet      0.0001 ( 99.9) 
34103 interpellaris    0.0001 ( 99.9) 
34104 deperat          0.0001 ( 99.9) 
34105 concinnator      0.0001 ( 99.9) 
34106 literatis        0.0001 ( 99.9) 
34107 promanant        0.0001 ( 99.9) 
34108 determinentur    0.0001 ( 99.9) 
34109 concertandi      0.0001 ( 99.9) 
34110 addiscas         0.0001 ( 99.9) 
34111 ponderes         0.0001 ( 99.9) 
34112 deglutire        0.0001 ( 99.9) 
34113 ruminaris        0.